# Labelling Pipeline (Label & Tag)
### Interactive Audio Annotation Tool

In [5]:
import librosa
import ipywidgets
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import pandas as pd
import glob, os
import matplotlib.gridspec as gridspec
import numpy as np
import re
os.chdir('/home/abdullah/avenging_forests/margalla_pilot')

try:
    from tkinter import Tk
    from tkFileDialog import askopenfilenames
except:
    from tkinter import Tk
    from tkinter import filedialog

from src.global_utils import vis_spectrogram1, get_labels
from src.footstep_utils import get_orig_fname, pred_thresholding
    
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)',text) ]

In [6]:
def get_wav_path():
    print('Please select the directory containing Audio WAV files ...')
    Tk().withdraw()
    wav_dir = filedialog.askdirectory()
    return wav_dir

def get_input_csv():
    print('Please select the CSV to read Data From!')
    Tk().withdraw()
    input_csv_path = filedialog.askopenfilename()
    print('>> Loading CSV: ', input_csv_path)
    df = pd.read_csv(input_csv_path)
    df['wav_time'], df['wav_counter'] = get_orig_fname(df['fname'])
    return df, input_csv_path
    
def initialize_labelled_csv(ans, fnames):
    if (ans == 'n') | (ans == 'N'):  # Load labelled_CSV
        print('Please Select the Labelled CSV to Continue from ...')
        Tk().withdraw()
        labelled_csv_path = filedialog.askopenfilename()
        labelled_df = pd.read_csv(labelled_csv_path)

        ## Dropping Already Labelled Samples from FNAMES
        drop_fnames = [os.path.basename(path) for path in labelled_df['path']]
        drop_index = fnames[fnames.isin(drop_fnames)].index.values
        fnames = fnames.drop(drop_index)
        fnames.index = list(range(len(fnames)))
        print('>> Dropped {} samples from fnames'.format(len(drop_index)))
        
    elif (ans == 'y') | (ans == 'Y') | (ans == ''): # Create NEW Labelled CSV
        labelled_df = pd.DataFrame(columns=['path', 'ts', 'tags', "comment", "diff", "set"])
        fname = input("What should be the name of the CSV file? [fname].csv")
        labelled_csv_path = "margalla_field_data/dn_9nov/{}.csv".format(fname)
    
    return labelled_df, labelled_csv_path
        
df, input_csv_path = get_input_csv()
labels = get_labels('footstep_detection')
thresh_df = pred_thresholding(df, labels, thresh=0.2)
dir_path = os.path.split(input_csv_path)[0]

wav_dir_path = get_wav_path()
sorted_paths = sorted(glob.glob(wav_dir_path + '/*.wav'), key=os.path.getmtime)
printlen(sorted_paths)
#Sorting File Paths here
wav_paths = [os.path.basename(wav) for wav in sorted_paths]
wav_paths.sort(key=natural_keys)
fnames = pd.Series(wav_paths)

labelled_response = input('Would you like to create a NEW labelled CSV file? [Y/n]')
labelled_df, labelled_csv_path = initialize_labelled_csv(labelled_response, fnames)
    
df.head()

Please select the CSV to read Data From!
>> Loading CSV:  /home/abdullah/avenging_forests/margalla_pilot/margalla_field_data/dn_9nov/footstep_pred_dn_9nov.csv
(37778, 82)
Please select the directory containing Audio WAV files ...


Would you like to create a NEW labelled CSV file? [Y/n] n


Please Select the Labelled CSV to Continue from ...
>> Dropped 0 samples from fnames


,fname,pred,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),...,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing),wav_time,wav_counter
0,4D-0-0.wav,Gurgling,0.001200,0.000263,0.000978,0.000894,0.002356,0.006656,0.002659,0.003576,...,0.016622,0.018431,0.006493,0.002720,0.001825,0.074099,0.000499,0.042276,4D,0
1,4D-0-1.wav,Gurgling,0.000236,0.000139,0.000557,0.001632,0.000798,0.008876,0.000828,0.002218,...,0.006313,0.002495,0.003460,0.000874,0.001205,0.008636,0.000086,0.003902,4D,0
2,4D-0-2.wav,Computer_keyboard,0.002126,0.000388,0.002872,0.002509,0.002321,0.005995,0.005898,0.001882,...,0.004369,0.004138,0.006195,0.001594,0.002628,0.015139,0.000608,0.006118,4D,0
3,4D-0-3.wav,Computer_keyboard,0.002351,0.000231,0.005428,0.001828,0.001904,0.004080,0.002265,0.001644,...,0.001094,0.003524,0.003645,0.001179,0.003262,0.067813,0.000665,0.012476,4D,0
4,4D-0-4.wav,Computer_keyboard,0.001150,0.000119,0.001435,0.001332,0.001868,0.001989,0.001572,0.001455,...,0.001518,0.002443,0.002222,0.001006,0.004514,0.036420,0.000445,0.010874,4D,0


In [10]:
glob.glob(wav_dir_path + '/*.wav')

[]

# 1. Audio Details  - Top Chunk

In [36]:
def get_fname_predictions(fname):
    preds = thresh_df[thresh_df['fname']==fname][labels]
    indexes = np.where((preds == 1).values == True)[1]
    sample_tags = [labels[idx] for idx in indexes]
#     display("==================    PREDICTIONS:    =============")
    prediction_string = " ".join([t for t in sample_tags])
    return prediction_string

def update_wav(fname):
    path = os.path.join(wav_dir_path, fname)
    audio, sr = librosa.load(path, sr=16000)
    vis_spectrogram1(audio, sr)
    display(plt.show())
    display(Audio(path))
    display(Audio(data=audio, rate=sr))
    
    predictions_widgets.value = get_fname_predictions(fname)
    fname_text_widget.value = fname


    
top_output = ipywidgets.Output(layout={'border': '1px solid black'})
fname_widget = ipywidgets.Dropdown(options=fnames[:15], description='Wav Name: ')
# def fname_handler(change):
#     display(change.new)
# fname_widget.observe(fname_handler, names='value')
    
load_audio_btn = ipywidgets.Button(description='Load Audio')
load_audio_btn.style.button_color = 'orange'
def btn_eventhandler(obj):
    top_output.clear_output()
    with top_output:
        update_wav(fname_widget.value)
load_audio_btn.on_click(btn_eventhandler)

fname_text_widget = ipywidgets.Text(value="0", description='Fname:', disabled=True)
predictions_widgets = ipywidgets.Text(value="0", description='Predictions:', disabled=True, layout=ipywidgets.Layout(width='50%', height='60px'))

fname_load_box = ipywidgets.HBox([fname_widget, load_audio_btn])
fname_description_box = ipywidgets.HBox([fname_text_widget, predictions_widgets])

final_top_box = ipywidgets.VBox([fname_load_box, top_output, fname_description_box])
final_top_box

## 2. Label & Tag - Mid Chunk

In [37]:
def extract_widget_labels():
    ts_values, tag_values = [], []
    
    print("TIME STAMPS: ")
    for i in range(NUM_TS_ROWS):
        ts_label_value = ts_label[i].value
        if ts_label_value in TS_LABELS:
            ts_values.append([ts_label_value, ts[i].value])
            print("\t",ts_label_value, ts[i].value)
    """
    for i in range(NUM_TAG_ROWS):
        if tags[i].value in MULTI_LABELS:
            tag_values.append(tags[i].value)
    """
    
    tag_values = list(tags.value)
    print("TAGS:", tag_values)
    
    return ts_values, tag_values
    
def clean_labels_df(raw_ts, raw_labels):
    print(raw_ts, raw_labels)
    clean_tags = " ".join([lab for lab in raw_labels])
    clean_ts = " ".join([" ".join([row[0], str(row[1][0]), str(row[1][1])]) for row in raw_ts])
    
    return clean_ts, clean_tags


##################      TimeStep Labelling WIDGETS Section       ##################
NUM_TS_ROWS = 3
TS_LABELS = ['Speech', 'Footsteps']
MULTI_LABELS = ['speech', 'footsteps', 'rain', 'birds', 'cricket', 'dn']

ts = {}
for i in range(NUM_TS_ROWS):
    ts[i] = ipywidgets.FloatRangeSlider(min=0, max=10.0, step=0.5, description='Time {}:'.format(i+1))

ts_label = {}
for i in range(NUM_TS_ROWS):
    ts_label[i] = ipywidgets.Combobox(placeholder='Label ???', options=TS_LABELS, description=' ==>>'.format(i+1))

# VIS LAYOUT
ts_rows = {}
for i in range(NUM_TS_ROWS):
    ts_rows[i] = ipywidgets.HBox([ts[i], ts_label[i]])
ts_section = ipywidgets.VBox([ts_rows[i] for i in range(NUM_TS_ROWS)])

####################################################################
############     Tagging WIDGETS Section     ############
tags = ipywidgets.SelectMultiple(
    options=MULTI_LABELS,
    #rows=10,
    description='Multi Labels',
    disabled=False
)

####################################################################
comment_box = ipywidgets.Text(placeholder='Any Comments ?', description='Comment:')
counter_widget = ipywidgets.IntText(
    value=0,
    description='Counter:',
    disabled=True
)

isTest_widget = ipywidgets.RadioButtons(options=['Test', 'Train', 'None'], value='Train', description='', disabled=False,
                                       layout=ipywidgets.Layout(width='90px', height='60px'))
diff_widget = ipywidgets.RadioButtons(options=['default', 'medium', 'hard'], value='default', description='', disabled=False)

tag_n_test_box = ipywidgets.HBox([tags, isTest_widget, diff_widget])
label_chunk = ipywidgets.VBox([ts_section, tag_n_test_box, ipywidgets.HBox([comment_box, counter_widget])])
label_chunk

## 3. Save & Navigate - Bottom Chunk

In [160]:
def save_labels(wav_widget_value):
    counter_widget.value = counter_widget.value + 1
    
    with output_bottom:
        global labelled_df
        print('Saving Labels')

        wav_path = os.path.join(wav_dir_path, fnames[fnames==wav_widget_value].values[0])
        new_df = pd.DataFrame(data={"path": wav_path}, index=[0])

        raw_ts, raw_labels = extract_widget_labels()

        new_df['comment'] = comment_box.value
        new_df['ts'], new_df['tags'] = clean_labels_df(raw_ts, raw_labels)
        new_df['set'] = isTest_widget.value
        new_df['diff'] = diff_widget.value
        
        old_shape = labelled_df.shape
        labelled_df = labelled_df.append(new_df, ignore_index=True)
        print("Old: {}   ==>>  New: {}".format(old_shape, labelled_df.shape))
        display(new_df)
    return labelled_df

output_bottom = ipywidgets.Output(layout={'border': '1px solid black'})


############################################################
   ##############        BUTTONS       ################
############################################################

###  1. NEXT / SAVE
next_btn = ipywidgets.Button(description='Next / Save', icon='check',button_style='success')
def next_btn_eventhandler(obj):
    output_bottom.clear_output()
    
    labelled_df = save_labels(fname_widget.value)
    
    new_idx = fnames[fnames==fname_widget.value].index[0] + 1
    try:
        fname_widget.value = fnames[new_idx]
    except:
        start_iter = max(0, new_idx)
        stop_iter = min(len(fnames), new_idx + 15)
        fname_widget.options = fnames[start_iter:stop_iter].values.tolist()
        
    with output_bottom:
        for i in range(NUM_TS_ROWS):
            ts_label[i].value = ""
        isTest_widget.value = "Train"
        diff_widget.value = "default"
        

    top_output.clear_output()
    with top_output:
        update_wav(fname_widget.value)
next_btn.on_click(next_btn_eventhandler)

###   2. CHECKPOINT
checkpoint_btn = ipywidgets.Button(description="Checkpoint", icon='check')
def chk_btn_event(obj):
    output_bottom.clear_output()
    with output_bottom:
        print('[Checkpoint REACHED]\nShape ==>>', labelled_df.shape)
        print(">> Save Path: ", labelled_csv_path)
        labelled_df.to_csv(labelled_csv_path, index=False)
checkpoint_btn.on_click(chk_btn_event)


###   3. JUMP
jump_mapper = {"5 min": 30, "10 min": 30*2, "20 min": 30*4}
select_jump = ipywidgets.Select(
    options=jump_mapper.keys(),
    description='',
    disabled=False
)
jump_btn = ipywidgets.Button(description="Jump")
def jump_btn_event(obj):
    output_bottom.clear_output()
    with output_bottom:
        jump_num = jump_mapper[select_jump.value]
        current_idx = fnames[fnames==fname_widget.value].index[0]
        new_idx = current_idx + jump_num
        print("Current IDX: {}  |  NEW IDX: {}  | Skip Num: {}".format(current_idx, new_idx, jump_num))

        start_iter = max(0, new_idx)
        stop_iter = min(len(fnames), new_idx + 15)
        fname_widget.options = fnames[start_iter:stop_iter].values.tolist()
        fname_widget.value = fnames[new_idx]
        
    top_output.clear_output()
    with top_output:
        update_wav(fname_widget.value)
jump_btn.on_click(jump_btn_event)

###   4. SKIP
skip_btn = ipywidgets.Button(description="SKIP", button_style='danger', icon="close")
def skip_btn_event(obj):
    output_bottom.clear_output()
    with output_bottom:
        print('Skipping/Trashing Sample!!')
    
    new_idx = fnames[fnames==fname_widget.value].index[0] + 1
    try:
        fname_widget.value = fnames[new_idx]
    except:
        start_iter = max(0, new_idx)
        stop_iter = min(len(fnames), new_idx + 15)
        fname_widget.options = fnames[start_iter:stop_iter].values.tolist()
        fname_widget.value = fnames[new_idx]
    
    top_output.clear_output()
    with top_output:
        print(fname_widget.value)
        update_wav(fname_widget.value)
skip_btn.on_click(skip_btn_event)

###   5. SEARCH
def search_target_index(search_by, search_string, curr_wav):
    if search_by is "Name":
        with output_bottom:
            try:
                search_index = fnames[fnames == search_string].index[0]
            except IndexError as err:
                print(err)
                print('ERROR: File not Found in "fnames" dataframe')
                search_index = fnames[fnames==curr_wav].index[0]
        
    if search_by is "Index":
        if (search_string.isdigit()):
            if (int(search_string) < len(fnames)):
                search_index = int(search_string)
            else:
                print("ERROR: in Index Search\nIndex greater than {}!".format(len(fnames)-1))
                search_index = fnames[fnames==curr_wav].index[0]    
        else:
            print("ERROR: in Index Search\nAlphabets detected in Search string")
            search_index = fnames[fnames==curr_wav].index[0]
    
    return search_index

search_option = ipywidgets.SelectMultiple(options=['Name', 'Index'], description='Search By:')#, value='Name')
search_text_box = ipywidgets.Text(placeholder='Name or Index', description='Search:')
search_btn = ipywidgets.Button(description="Search", button_style='primary', icon='search')
def search_btn_event(obj):
    output_bottom.clear_output()
    
    with output_bottom:
        search_by, search_string = search_option.value[0], search_text_box.value
        search_index = search_target_index(search_by, search_string, fname_widget.value)
        
        try:
            fname_widget.value = fnames[search_index]
        except:
            start_iter = max(0, search_index)
            stop_iter = min(len(fnames), search_index + 15)
            print('[INFO] Reached Dropdown Limit:', fnames[start_iter:stop_iter].values.tolist()[0])
            print(start_iter, stop_iter)
            
#             last_label = labelled_df['path'].values[-1:][0].split('/')[-1]
#             last_label_num = last_label[-1].split('.')[0]
#             last_label[-1] = str(int(last_label_num) + 1) + '.wav'
#             new_label = "-".join(last_label)
#             new_index = fnames.tolist().index(new_label)
            
#             fname_widget.options = fnames[new_index:new_index + 15].values.tolist()
            fname_widget.options = fnames[start_iter:stop_iter].values.tolist()
            
        
        print('Index: {}   |  Fname: {}'.format(search_index, fname_widget.value))
        
    top_output.clear_output()
    with top_output:
        update_wav(fname_widget.value)
search_btn.on_click(search_btn_event)

search_section = ipywidgets.HBox([search_option, search_text_box, search_btn])
buttons_line = ipywidgets.HBox([checkpoint_btn, next_btn, skip_btn, select_jump, jump_btn])
bottom_box = ipywidgets.VBox([buttons_line, search_section, output_bottom])
# bottom_box

## IMPLEMENTATION

In [161]:
final_layout = ipywidgets.VBox([final_top_box, label_chunk, bottom_box])
final_layout

## Urgent Fixes
1. Add Clear Buttons for TS labels & comments
2. Debug Skip Button
3. Display Prediction & FileName in Text Box
4. Audios Side by side

In [155]:
last_file_num = last_file_name[-1].split('.')[0]

In [156]:
last_file_name[-1] = str(int(last_file_num) + 1) + '.wav'

In [157]:
new_file_name = "-".join(last_file_name)

In [158]:
fnames.tolist().index(new_file_name)

1275

In [59]:
ts_label[0]

Combobox(value='', description=' ==>>', options=('Speech', 'Footsteps'), placeholder='Label ???')

In [66]:
ts_label[0].value = ""

In [10]:
from sys import executable, argv
from subprocess import check_output
from PyQt5.QtWidgets import QFileDialog, QApplication

def gui_fname(directory='../'):
    """Open a file dialog, starting in the given directory, and return
    the chosen filename"""
    # run this exact file in a separate process, and grab the result
    file = check_output([executable, 'run.sh', directory])
    return file.strip()


In [14]:
try:
    from tkinter import Tk
    from tkFileDialog import askopenfilenames
    print('A')
except:
    from tkinter import Tk
    from tkinter import filedialog
    print('B')

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filenames = filedialog.askopenfilenames() # show an "Open" dialog box and return the path to the selected file

print (filenames)


B
()
